# Agent生命周期钩子（Hooks）

## Motivation
当多个Agent运行以及任务流转中，会有以下需求：
- 调试需求：获取Task交接给哪个Agent？Agent在调用工具之前，它的“思考”是什么？工具返回的结果又是什么？
- 监控和日志：我想记录下每个Agent的启动和结束时间，以及它处理了哪些任务，用于性能分析或审计。
- 扩展功能：我希望在Agent开始处理任务前，预先加载一些特定的数据；或者在Agent结束后，将结果保存到数据库。

它提供了一种非侵入式的方式，让你能够在Agent运行过程中的关键节点（比如开始、结束、调用工具、交接任务）“挂载”你自己的代码。
简而言之，相当于业务流程的中的数据埋点作用，用户业务执行中出现的转化率断点。去发现Agent在运行过程中的一些问题，并且可以在问题发生时及时通知开发者。

## Definition：Hooks的类型
### AgentHooks
- 定义：类；继承并且重写感兴趣的事件方法（如`on_start`, `on_tool_end`等）。然后，在创建具体的Agent实例时，通过`hooks=`参数将这个钩子类的实例挂载上去

- 作用范围：仅对挂载了该钩子的Agent实例生效。其他Agent实例不会受到影响
- 相当于给某个员工（Agent）安装了工作记录仪

### RunHooks
- 定义：类；继承并且重写感兴趣的事件方法（方法名略有不同，如`on_agent_start`, `on_agent_end`）。但是它是用在调用`Runner.run()`或`Runner.run_sync()`时，通过`hook=`参数传递进去的

- 作用范围：对本次运行涉及的所有Agent都生效。无论任务流经多少Agent，只要发生了对应的事件，这个钩子的方法就会触发
- 比喻：在整体项目（Run）的入口处安排一个“全局观察员”，记录所有员工（Agent）的行为

| 事件       | AgentHooks 方法名 | RunHooks 方法名  | 触发时机                     | 关键参数 (除了context, agent)                          |
|------------|-------------------|------------------|------------------------------|-------------------------------------------------------|
| Agent开始  | on_start          | on_agent_start   | Agent开始处理任务            | 无                                                    |
| Agent结束  | on_end            | on_agent_end     | Agent完成任务并产生输出      | output (Agent的最终输出)                              |
| 任务交接   | on_handoff        | on_handoff       | 任务从一个Agent交接给另一个  | source (源Agent) (AgentHooks) / from_agent, to_agent (RunHooks) |
| 工具使用开始| on_tool_start     | on_tool_start    | Agent准备调用一个工具        | tool (被调用的工具对象)                               |
| 工具使用结束| on_tool_end       | on_tool_end      | Agent成功调用工具并获得结果  | tool (被调用的工具对象), output/result (工具返回的结果字符串) |

- 注意：`on_handoff`的参数在两类钩子中是不同的，`RunHooks`提供了更清晰的`form_Agent`和`to_agent`

## Design & Implementation：解析教程的代码

### AgentHooks
示例：监控Agent的开始与结束
```python
# ... (LLM配置等代码)

# 1. 定义继承自 AgentHooks 的类
class MyAgentHooks(AgentHooks):
    def __init__(self):
        self.event_counter = 0 # 用于计数事件发生顺序

    # 2. 重写 on_start 方法
    async def on_start(self, context: RunContextWrapper, agent: Agent) -> None:
        self.event_counter += 1
        print(f"{self.event_counter}: Agent {agent.name} started")

    # 3. 重写 on_end 方法
    async def on_end(self, context: RunContextWrapper, agent: Agent, output) -> None:
        self.event_counter += 1
        print(f"{self.event_counter}: Agent {agent.name} ended with output {output}")

# 4. 创建Agent时，将钩子实例传入 hooks 参数
agent = Agent(name="旅行助手", model=llm, hooks=MyAgentHooks(), instructions="You are a helpful assistant")

# ... (运行Agent的代码)
```

- `text:RunContextWrapper`:为上下文对象，包含当前运行的一些元信息；即使钩子函数用不到它，也必须保留这个参数，这是框架的要求。

### RunHooks
示例：全局监控Agent的开始与结束
```python
# ... (LLM配置等代码)

# 1. 定义继承自 RunHooks 的类
class MyRunHooks(RunHooks):
    def __init__(self): # 初始化class的变量
        self.event_counter = 0

    # 2. 重写 on_agent_start 方法 (注意方法名变化)
    async def on_agent_start(self, context: RunContextWrapper, agent: Agent) -> None:
        self.event_counter += 1
        print(f"{self.event_counter}: Agent {agent.name} started")

    # 3. 重写 on_agent_end 方法 (注意方法名变化)
    async def on_agent_end(self, context: RunContextWrapper, agent: Agent, output) -> None:
        self.event_counter += 1
        print(f"{self.event_counter}: Agent {agent.name} ended with output {output}")

# 4. 创建Agent时，不传入hooks
agent = Agent(name="旅行助手", model=llm, instructions="You are a helpful assistant")

async def main():
    # 5. 调用Runner.run时，将钩子实例传入 hooks 参数
    result = await Runner.run(agent, hooks=MyRunHooks(), input="孟子全名叫什么?")
    print(result.final_output)

# ... (运行代码)
```
流程：
- 调用`Runner.run()`或`Runner.run_sync()`时，框架会检查`agent`是否有`hooks`
- Agent开始工作前，调用`MyAgentHooks`的`on_start`方法;
- Agent产出最终`output`后，调用`MyAgentHooks`的`on_end`方法;

### RunHooks
示例：全局监控Agent的开始与结束

```python
# ... (LLM配置等代码)

# 1. 定义继承自 RunHooks 的类
class MyRunHooks(RunHooks):
    def __init__(self):
        self.event_counter = 0

    # 2. 重写 on_agent_start 方法 (注意方法名变化)
    async def on_agent_start(self, context: RunContextWrapper, agent: Agent) -> None:
        self.event_counter += 1
        print(f"{self.event_counter}: Agent {agent.name} started")

    # 3. 重写 on_agent_end 方法 (注意方法名变化)
    async def on_agent_end(self, context: RunContextWrapper, agent: Agent, output) -> None:
        self.event_counter += 1
        print(f"{self.event_counter}: Agent {agent.name} ended with output {output}")

# 4. 创建Agent时，不传入hooks
agent = Agent(name="旅行助手", model=llm, instructions="You are a helpful assistant")

async def main():
    # 5. 调用Runner.run时，将钩子实例传入 hooks 参数
    result = await Runner.run(agent, hooks=MyRunHooks(), input="孟子全名叫什么?")
    print(result.final_output)

# ... (运行代码)
```
流程：
- 当调用`Runner.run()`或`Runner.run_sync()`时，框架会检查`agent`是否有`hooks`
- 如果有，就会调用`MyRunHooks`就行监控这次运行中所有Agent的生命周期
- 当`旅行助手`Agent开始时触发`on_agent_start`方法
- 当`旅行助手`Agent产出最终`output`后，触发`on_agent_end`方法
- 如果任务发生交接，也会触发那些Agent钩子

>区别：
>- `AgentHooks`是为了监控单个Agent的生命周期，代码实现是写在`Agent`类中的
>- `RunHooks`是为了监控整个运行过程中所有Agent的生命周期，代码实现是写在`Runner`类中的

### Tools
如以上示例相似，不补充代码。
在`AgentHooks`与`RunHooks`中，只是将重写的方法换成了`on_tool_start`和`on_tool_end`。关键在于理解参数：
- `on_tool_start(..., tool: Tool)`：可以让你知道Agent打算调用哪个工具 (`tool.name`)。
- `on_tool_end(..., tool: Tool, output: str)`：可以让你知道工具实际执行后返回了什么结果 (`output`)。

这对于调试Agent与工具的交互（比如参数是否正确传递，工具返回是否符合预期）非常有用。

>教程存在问题，根据不同模型使用，偶尔LLM会不主动去调用工具，则导致无法实现良好的演示效果
>建议修改prompt：(可以提交个pr看看意见)
```python
    instructions="""
    始终用汉赋的形式回答用户;
    IMPORTANT RULE:
    1. 只使用get_weather工具来获取天气信息;
    2. 回答用户时，必须包含天气信息;
    """,
```

### 任务交接（监控Handoff）

`AgentHooks` (`on_handoff(..., agent: Agent, source: Agent)`)：
- `agent`: 指的是接收任务的Agent。
- `source`: 指的是发起交接的Agent。
- 这个钩子挂载在哪个Agent上，就在哪个Agent被交接给或发起交接时触发（具体触发逻辑可能需要查看源码或进一步实验确认，但至少能捕获到事件）。教程的例子是将钩子挂载到了所有Agent上，所以能看到`Triage Agent`交接给`Math Tutor`或H`istory Tutor`的事件。

```python
# ... (LLM配置等代码)

# 定义钩子，这里只定义了on_handoff这个事件
class MyAgentHooks(AgentHooks):
    def __init__(self):
        self.event_counter = 0

    async def on_handoff(self, context: RunContextWrapper, agent: Agent, source: Agent) -> None:
        self.event_counter += 1
        print(
            f"{self.event_counter}: Agent {source.name} handed off to {agent.name}"
        )

# ... 创建多个Agent

# ... (运行代码)
```

`RunHooks` (`on_handoff(..., from_agent: Agent, to_agent: Agent)`)：
- `from_agent`: 清晰地表示发起交接的Agent。
- `to_agent:` 清晰地表示接收任务的Agent。
- 这个钩子挂载在`Runner.run`上，所以能全局捕获任何Agent之间的交接事件，参数也更明确。

```python
# ... (LLM配置等代码)

# 定义钩子，这里只定义了on_handoff这个事件
class MyRunHooks(RunHooks):
    def __init__(self):
        self.event_counter = 0

    async def on_handoff(
        self, context: RunContextWrapper, from_agent: Agent, to_agent: Agent
    ) -> None:
        self.event_counter += 1
        print(
            f"### {self.event_counter}: Handoff from {from_agent.name} to {to_agent.name}."
        )

# ... 创建多个Agent

# ... (运行代码)
```

## Application Scenarios
- 详细日志记录：使用`logging`库记录Agent启动、结束、工具调用、交接等信息写入日志文件，用于事后分析。
- 性能监控：在`on_start`和`on_end`中记录时间戳，计算Agent运行时间
- 状态更新：在钩子函数中更新外部系统的状态，例如UI中显示“Agent正在思考...“或”正在调用天气API...“。
- 数据预取/后处理：在`on_start`时根据任务类型预加载所需数据，在`on_end`时根据任务类型后处理数据，例如保存到数据库、发送通知等。
- 调试复杂流程：通过打印`on_handoff`中的参数，你可以清晰地看到Agent之间的交接流程，帮助你调试复杂的任务分配逻辑；Tool同理。

## 局限性
- 异步要求：所有钩子方法都必须是`async def`定义的协程，因为Agent的执行本身是异步的
- 性能考虑：钩子函数中的代码不宜过于耗时
- 信息有限：钩子提供的上下文信息（`context`对象）目前比较有限